#  CNN
畳み込み層 は入力データの一部分に注目しその部分画像の特徴を調べる層である。
どのような特徴に注目すれば良いかは、学習用のデータや損失関数等を適切に定めることによって自動的に学習される。 
例えば顔認識をするCNNの場合適切に学習が進むと、 入力層に近い畳み込み層では線や点といった低次元な概念の特徴に、出力層に近い層では目や鼻といった高次元な概念の特徴に注目するようになる 。 
注目すべき特徴はプログラム内部では フィルター(カーネル) と呼ばれる重み行列として扱われ、各特徴につき一つのフィルターを用いる。
#  プーリング層
プーリング層はのように 畳み込み層の出力を縮約しデータの量を削減する層と言える。
特徴マップの部分区間の最大値を取ったり（ Maxプーリング ）、あるいは平均を取ったり（ Averageプーリング ）することでデータの圧縮を実現する。
畳み込みを行うと、画像の中の特徴量の分布を調べることができるが、同じ特徴は同じような場所にかたまって分布していることが多く、また時に特徴が見つからない場所が広く分布していることもあるなど、畳み込み層から出力される特徴マップにはそのデータの大きさに対して無駄が多くある。

プーリングはそのようなデータの無駄を削減し、情報の損失を抑えながらデータを圧縮することができる。 
その反面、プーリングによって細かい位置情報は削除されてしまうが、逆にこれは、プーリング層によって抽出された特徴が元画像の平行移動などでも影響を受けないようなロバスト性を与える役割を果たす。例えば、写真に映る手書き数字の分類を行う場合、数字の位置は重要ではないが、プーリングによってそのようなあまり重要でない情報を削除し、入力画像に対する被検出物の位置の変化に強いモデルを構築することができる

In [1]:
import numpy as np
import pandas as ps
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
sns.set 

/Users/ranmarusato/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/ranmarusato/.matplotlib/matplotlibrc", line #629
  (fname, cnt))


<function seaborn.rcmod.set>

In [2]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)または(n_samples,)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード

    Retruns
    ----------
    for文で呼び出すと以下の2つを返す。最後のイテレーションでは、バッチサイズより小さいこともある。
    mini_X : 次の形のndarray, shape (batch_size, n_features)
      学習データのミニバッチ
    mini_y : 次の形のndarray, shape (batch_size, 1)または(batch_size,)
      正解値のミニバッチ
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        # ランダムに並べ換える
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X.iloc[shuffle_index]
        self.y = y.iloc[shuffle_index]
        self._counter = 0
        # イテレーション数を計算する
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        # len()が使われたときの処理
        return self._stop

    def __iter__(self):
        return self

    def __next__(self):
        # for文で呼ばれた際のループごとの処理
        if self._counter >= self._stop:
            # 最後まで進んだら終了
            self._counter = 0 # カウンターをリセット
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [3]:
class SimpleInitializer:
    def __init__(self):
        self.sigma = 0.01
        
    def W(self, n_nodes1, n_nodes2):
        W = self.sigma * np.random.randn( n_nodes1, n_nodes2)
        return W
    

    def B(self, n_nodes2):
        """
    バイアスの初期化
    Parameters
    ----------
    n_nodes2 : int
      後の層のノード数

    Returns
    ----------
    B :
        """
        B = self.sigma * np.random.randn(n_nodes2,)
        return B

In [4]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        #重みのサンプル数分 shapeで割る
        
        layer.W -= self.lr *  layer.dW/layer.dW.shape[0]
        #バイアス項のサイズ感を注意
        #サンプル数、サンプル数で割る
        #　サンプル数を保持している変数
        layer.B -=  self.lr*layer.dB.mean()
        
        return layer

In [5]:
class Identity_function_MSE:
    def forward(self,y,pred):
        mse = np.mean((y-pred) **2)/2
        return mse
        
    def backwarod(self,y,pred):
        delta = y - pred
        return 


In [6]:
class identify_function_MSE:
    def forward(self,y,pred):
        return np.mean((y - pred) **2)/2
    def backwarod(self,y,pred):
        return y - pred

In [7]:
class tanh:
    
    def forward(self,x):
        self.A = np.tanh(x)
        return self.A
        
    def backward(self,x):
        
        return (1 - np.tanh(self.A)**2 )

# 全結合層のクラス化

In [8]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 =  n_nodes1
        self.n_nodes2 = n_nodes2
        
        #引数として、initializerクラスのインスタンス変数とを受け取る、
        
        self.W = initializer.W(self.n_nodes1,self.n_nodes2)
        self.B  =initializer.B(self.n_nodes2,)
        self.X = None
        self.dW = None
        self.dB = None
        # 初期化
        
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """       
        self.X = X
        A = np.dot(X,self.W) + self.B
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        # 更新
        #全てのメンバ変数を
        #次に渡す勾配、この勾配をベースに、その層での重みやバイアス項が計算される
        dZ = np.dot(dA,self.W.T)
        
        #次に渡す重みのサイズ
        self.dW = np.dot(self.X.T,dA)
        
        #バッチ処理に対応したバイアス項
        self.dB =  dA
        #更新
        # self.optimizer.update(self)でも更新される
        self = self.optimizer.update(self)

        return dZ

#  Trainerクラス

In [9]:
class Trainer:
    def __init__(self, n_epochs, n_nodes1, n_nodes2, n_output,batch_size = 10,lr=0.001):
        self.n_epochs = n_epochs
        #バッチサイズ
        self.batch = batch_size 
        #学習率
        self.lr = lr
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.sigma = None
        #アトリビュート
        self.loss = []
        self.cost  = []
        self.valloss = []

    def fit(self,X_train,y_train):
        #特徴量を取得
        n_features = X_train.shape[1]
        gb = GetMiniBatch(X_train, y_train,batch_size=10)
        optimizer = SGD(self.lr)
        self.FC1 = FC(n_features, self.n_nodes1, SimpleInitializer(self.sigma), optimizer)
        self.activation1 = tanh()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), optimizer)
        self.activation2 = tanh()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), optimizer)
        self.activation3 = Identity_function_MSE()
         
        for i in range(self.n_epochs):
            count = 0
            cost = 0
            for mini_X_train, mini_y_train in gb:
                #イテレーション毎のフォワード
                mini_y_train = mini_y_train[:,np.newaxis]

                A1 = self.FC1.forward(mini_X_train)
                Z1 = self.activation1.forward(A1)       

                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                
                A3 = self.FC3.forward(Z2)

                Z3 = A3
                 
                loss = self.activation3.forward(Z3,mini_y_train)
                count += 1
                
                cost += loss
                self.loss.append(loss)
                #逆伝播で次に渡す勾配を計算
                dA3 = Z3 -  mini_y_train
                #全結合層の逆伝播
                dZ2 = self.FC3.backward(dA3)
                #活性化関数の層の逆伝播
                dA2 = self.activation2.backward(dZ2)
                
                dZ1 = self.FC2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                
                dZ0 = self.FC1.backward(dA1) # dZ0は使用しない


        return self.loss
    


    
    def predict(self,X):
        # 1層目  ############################################
        print(self.FC1.dW.shape)
        rayer1_pre = np.dot(X, self.FC1.dW) 
        rayer1_out = np.tanh(rayer1_pre)

        # 2層目  ############################################
        rayer2_pre = np.dot(rayer1_out,  self.FC2.dW) 
        rayer2_out = np.tanh(rayer2_pre)

        # 3層目  ############################################
        pred = np.dot(rayer2_out, self.FC3.dW)

        return pred

In [10]:
X = pd.read_csv('train.csv')
y = X['SalePrice']
X = X.loc[:,['GrLivArea','YearBuilt']]

In [11]:
y = np.log(y)

In [12]:
model = Trainer(10,300,100,1)

In [15]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

# データの読み込み

In [16]:
mnist_dir = "mnist_data/"

In [17]:
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)

In [18]:
# trainとtestに分割する
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.14285, shuffle=False)
# ラベルをint型にしておく
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

In [19]:
X_train[1].shape

(784,)

In [20]:
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])

a = np.empty((2, 3))

indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)


In [21]:
a[0] = x[indexes0]*w

In [22]:
 x[indexes0]

array([1, 2, 3])

In [23]:
a[0]

array([ 3., 10., 21.])

In [24]:
 x[indexes1]

array([2, 3, 4])

In [25]:
a[1] = x[indexes1]*w

In [26]:
a[1] 

array([ 6., 15., 28.])

In [27]:
a = a.sum(axis=1)

In [28]:
a

array([34., 49.])

In [ ]:
class Conv2d:
    def __init__(self):
    def forward(self):
    def backward(self):
    

# 2d Convolution  Forward

In [378]:
def output_size(height, width, padding, filter_height, filter_width, stride):
    output_height = (height + 2*padding - filter_height)//stride +1
    output_width= (width + 2*padding - filter_width)//stride +1
    output_height =int( output_height)
    output_width =int( output_width)
    return [output_height,output_width]

In [394]:
padding =0

In [393]:
np.pad(X_train,((0,0),(0,0),(padding,padding),(padding,padding)),'constant',constant_values=0).shape

(60000, 1, 32, 32)

In [ ]:
a = np.zeros([2,])

In [603]:
X_train.shape

(60000, 1, 28, 28)

In [604]:
X_train_pad = np.pad(X_train,((0,0),(0,0),(padding,padding),(padding,padding)),'constant',constant_values=0)

In [605]:
X_train_pad.shape

(60000, 1, 28, 28)

In [654]:
a = np.zeros([30,1,26,26])

In [658]:
f_height=3
f_width = 3

stride =1
a_list =[]
F = 1
padding = 0
W =np.random.randint(0,9,(1,3,3))
num, channel, width, height = X_train.shape
#サンプル数
output_shape = output_size(height,width,padding,f_height,f_width,stride)
# a = np.zeros([30,output_shape[0],output_shape[1]])
count = 0
X_train_pad = np.pad(X_train,((0,0),(0,0),(padding,padding),(padding,padding)),'constant',constant_values=0)

for  n in range(30):
    #フィルター   
    for f  in range(F):
        count_w_index = 0
        count_h_index = 0  
        #幅:w
        for  i in range(0, (output_shape[0]), stride):
                
#             print(i)
#             print(n)
#             print(c)
#             print(F_SIZE)
#             print(count_w_index)             a[count_w_index, count_h_index] =   (X_train[n,c][i:i+F_SIZE, 0:F_SIZE]*W).sum()
#             print(count_w_index, count_h_index)      
#             print(count_h_index)              
#             #高さh
            for s in range(0, output_shape[1], stride):
#                 print(i,i+F_SIZE, s,s+F_SIZE)#インデックスのデバッグ
#                 print ((X_train[n,c][i:i+F_SIZE, s:s+F_SIZE]*W).sum())
#                 print(X_train_pad[n,c][i:i+f_height, s:s+f_width ])
                #バイアスはあとでたす
#                 a[n,i,s] += (X_train_pad[n,c,i:i+F_SIZE,s:s+F_SIZE]*W[f]).sum() 
#                 print(i,i+F_SIZE)
#                 print((X_train_pad[n,c,i:i+f_height,s:s+f_width]*W[f]))
#                 print((X_train_pad[n,c,i:i+f_height,s:s+f_width]*W[f]))
#                 print(np.sum(X_train_pad[n,c,i:i+f_height,s:s+f_width]*W[f]))
                a[n,f,int(i/stride),int(s/stride)] += (np.sum(X_train_pad[n,:,i:i+f_height,s:s+f_width]*W[f])).sum() 
#                 print('インデックス')
#                 print(i,s )
#                 print(count_w_index, count_h_index)  

In [659]:
a.shape

(30, 1, 26, 26)

In [650]:
a.reshape(30,1,26,26).shape

(30, 1, 26, 26)

# 2d convolution backward

In [695]:
N, C, H, W = X_train.shape

In [717]:
W.shape

(1, 1, 3, 3)

In [672]:
F,C,FH,FW = W.shape

In [675]:
output_shape = output_size(X_train.shape[2],X_train.shape[3],0,W.shape[2],W.shape[3],stride)

In [678]:
output_shape

[26, 26]

In [685]:
W[0,:,:,:]

array([[[2, 0, 1],
        [2, 2, 0],
        [0, 2, 0]]])

In [703]:
W

array([[[[1, 0, 1],
         [0, 2, 0],
         [0, 0, 0]]]])

In [709]:
X_train.shape

(60000, 1, 28, 28)

In [304]:
A[0]

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,   57.,  300.,  496.,  567.,  330.,  134.,    6.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
         119.,  414.,  910., 1240., 1327., 1017.,  578.,  196.,   10.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    9.,   81.,
         435.,  974., 1651., 1999., 2087., 1777., 1279.,  696.,  257.,
          53.,    0.,    0.,  

In [305]:
A[4]

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,   57.,  300.,  496.,  567.,  330.,  134.,    6.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
         119.,  414.,  910., 1240., 1327., 1017.,  578.,  196.,   10.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    9.,   81.,
         435.,  974., 1651., 1999., 2087., 1777., 1279.,  696.,  257.,
          53.,    0.,    0.,  

In [291]:
A[8]

array([[  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
          0. ,   5.7,  30. ,  49.6,  56.7,  33. ,  13.4,   0.6,   0. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         11.9,  41.4,  91. , 124. , 132.7, 101.7,  57.8,  19.6,   1. ,
          0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0.9,   8.1,
         43.5,  97.4, 165.1, 199.9, 208.7, 177.7, 127.9,  69.6,  25.7,
          5.3,   0. ,   0. ,  

# Max プーリング

In [491]:
A = np.zeros([10,14,14])
B = np.zeros([10,14,14])
C= np.zeros([10,14,14])

In [467]:
B

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [468]:
 output_size(28,28,0,2,2,2)

[14, 14]

In [475]:
C[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [563]:
r = np.zeros([2,])

In [578]:
#プーリングの高さと幅
f_height=2
f_width = 2
F_SIZE = 2
stride =2
a_list =[]
F = 1
padding = 0
num, channel, width, height = X_train.shape
#サンプル数
output_shape = output_size(height,width,padding,f_height,f_width,stride)
count = 0
index_list = []
column_list = []
X_train_pad = np.pad(X_train,((0,0),(0,0),(padding,padding),(padding,padding)),'constant',constant_values=0)
for  n in range(1):
    for f  in range(F):
        count_w_index = 0
        count_h_index = 0  
        #幅:w
        for  i in range(0, (output_shape[0]), stride):
#             print(count_w_index)             a[count_w_index, count_h_index] =   (X_train[n,c][i:i+F_SIZE, 0:F_SIZE]*W).sum()
#             print(count_w_index, count_h_index)      
#             print(count_h_index)            
#             #高さh
            for s in range(0, output_shape[1], stride):
#                 print(i,i+F_SIZE, s,s+F_SIZE)#インデックスのデバッグ
#                 print ((X_train[n,c][i:i+F_SIZE, s:s+F_SIZE]*W).sum())
#                 print(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE])
#                 print(np.max(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]))
                #最大値を取得し対応する代入
                A[n,i,s] = (np.max(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]))
                #最大値のインデックスを取得
#                 print((np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE])))
                B[n,i,s]  = (np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]))      
#                 print(np.unravel_index(np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE](f_height,f_width))))
#                 print(np.unravel_index(np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]),(f_height,f_width)))
                index,colum = np.unravel_index(np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]),(f_height,f_width))      
                #最大値のインデックスを保存
                index_list.append(index)
                column_list.append(colum)
                #np.array(np.unravel_index(np.argmax(X_train_pad[n,c][i:i+F_SIZE, s:s+F_SIZE]),(f_height,f_width)))+np.array([i,s])

In [534]:
D = np.zeros([1,1414])

In [519]:
X_train_pad[0,0].shape

(28, 28)

In [514]:
np.array([1,1]) + np.array([1,1])

array([2, 2])

In [500]:
X_train[1]

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,  64, 253, 255,  63,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,  96, 205, 251, 253, 205, 111,   4,   0,
           0,   0,   0

In [472]:
C, N 


(10, 14, 14)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [487]:
C

In [488]:
type(C)

list

# Flatten(平滑化)

In [486]:
type(C)

numpy.ndarray

In [43]:
model = SimpleConv1d()

In [44]:
model.forward(X).reshape(1,2)

array([[35, 50]])

In [45]:
X = np.array([1,2,3,4])

In [46]:
model.backward(X)

(array([30]), array([ 50,  80, 110]), array([ 30, 110, 170, 140]))

In [47]:
np.arange(1,5,1)

array([1, 2, 3, 4])

In [48]:
A = np.zeros(222)

In [49]:
X.shape[0]

4

In [50]:
out = out_put(X.shape[0],3)

In [51]:
out

2

In [52]:
def forward(X,W,output_width):
    """
    フォワード
    Parameters
    ----------
    X : 次の形のndarray, shape (batch_size, n_nodes1)
        入力
    Returns
    ----------
    A : 次の形のndarray, shape (batch_size, n_nodes2)
        出力
    """       
    a_list = []
    count = 0
    #1*3のフィルターを定義
    initialize = SimpleInitializer(0.01)
    B = np.array(1)
    for i in range(len(X)-1):  
        #出力のセル１個分
        r = 0
        for s in range(W.shape[0]):  
            if i +s >= X.shape[0]:
                 return np.array(a_list)
            r += np.dot(X[i+s],W[s])
            #バイアス項を加算
        r += B
        a_list.append(r)

    
    

In [53]:
def backward(X,W,delta_a):
    delta_b  = np.sum(delta_a,axis =0,keepdims=True)
    #Wの勾配を計算
    for i  in range(W.shape[0]):
        index = np.arange(i,i+2)
        print(np.dot(delta_a,X[index]))
    #Xの勾配を計算
    for i  in range(X.shape[0]):
    #インデックスがXのはじめと、末尾のみ別の処理
        if i == 0 :
            print(delta_a[i]*W[i])
        elif i == len(X)-1:
            print(delta_a[i-2]*W[i-1])
        else:
            index = np.array([i,i-1])
            index = np.array(index)
            print(np.dot(delta_a,W[index]))
    

# データセットの用意

In [302]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

# 保存先を指定
mnist_dir = "./mnist_data/"

# MNISTの読み込み
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)
# trainとtestに分割する
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.14285, shuffle=False)
# ラベルをint型にしておく
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

In [303]:
X_train.shape

(60000, 784)

#  ２次元に変換

In [41]:
X_test = X_test.reshape(10000,1,28,28)

In [42]:
X_train = X_train.reshape(60000,1,28,28)